In [31]:
import pandas as pd 
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [32]:
bestDesc = pickle.load(open('./data/bestDescV2.pkl', 'rb'))
train = pickle.load(open('./dtTrain.pkl', 'rb'))
test = pd.read_csv("data/TestSetNoId.csv")
bestDesc

['TDB2i', 'ATSC8i', 'VR1_D.1', 'AATS8i.1', 'VR2_Dzv.1', 'minHBint8']

In [33]:
dfTrain = train.loc[:,bestDesc]
dfTest = test.loc[:,bestDesc]

dfTrain.shape, dfTest.shape

((74, 6), (19, 6))

In [34]:
# 0. Preparation Data
x_train = dfTrain.iloc[:,:]
x_test = dfTest.iloc[:,:]
y_train = train.iloc[:, [-1]]
y_test = test.iloc[:,[-1]]

In [35]:
# Feature Scaler using standard scaler
# sc = StandardScaler()
# scale_x_train = sc.fit_transform(x_train)
# scala_x_test = sc.transform(x_test)

In [36]:
# Feature Scaler
# Using MinMaxScaler()
# scaler = MinMaxScaler()
# scaler.fit(x_train)
# scale_x_train = scaler.transform(x_train)
# scale_x_test = scaler.transform(x_test)

In [37]:
param_grid={
            'C': [0.1, 1, 10, 100, 1000],
            'gamma': ['auto','scale'],
}

In [38]:
gsc = GridSearchCV(
        estimator=SVR(kernel='rbf'),
        param_grid=param_grid,
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

gsc.fit(x_train, y_train)

c:\users\muhammad fajar rizqi\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': ['auto', 'scale']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [41]:
gsc.best_params_


{'C': 1000, 'gamma': 'scale'}

In [30]:
# View the accuracy score
print('Best score:', gsc.best_score_) 

Best score: -0.2999115194933487


In [42]:
# View the best parameters for the model found using grid search
print('Best C:',gsc.best_estimator_.C) 
print('Best Kernel:',gsc.best_estimator_.kernel)
print('Best Gamma:',gsc.best_estimator_.gamma)

Best C: 1000
Best Kernel: rbf
Best Gamma: scale


In [43]:
model = SVR(C=gsc.best_params_['C'],kernel="rbf", gamma=gsc.best_params_['gamma'])
model

SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [44]:
model.fit(x_train, y_train)

c:\users\muhammad fajar rizqi\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [45]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
y_train_pred, y_test_pred

(array([5.44913635, 4.85114143, 6.02422193, 6.45632091, 7.96356702,
        7.94811649, 6.87027513, 6.30654972, 8.0295483 , 6.76347604,
        8.05560768, 7.64420098, 6.04631393, 6.62799141, 6.63648084,
        6.14261018, 8.04086527, 7.83544778, 8.04639646, 5.83073552,
        8.04086527, 7.82089622, 7.56174068, 7.64312706, 7.62760926,
        6.06297547, 6.09397267, 8.12858827, 7.54910365, 6.5489151 ,
        7.87500836, 6.51579232, 6.530896  , 6.0191786 , 6.38558428,
        7.84496341, 6.9903162 , 7.1424508 , 6.95085541, 7.93402412,
        7.67835097, 5.81697997, 6.98381311, 5.94448013, 7.68936267,
        7.44101159, 6.76751168, 6.41632847, 8.23300765, 6.71161702,
        7.42287197, 7.18308683, 7.71241697, 6.25681509, 8.06525793,
        6.79924016, 7.95134028, 7.49284206, 5.82464251, 7.00298335,
        7.85872128, 8.05495012, 8.09124072, 7.66673333, 6.76134209,
        7.05883589, 6.85920766, 7.9422124 , 6.63318194, 8.01890255,
        7.28258569, 7.78714629, 8.13791962, 5.82

In [46]:
r2_train = r2_score(y_train,y_train_pred)
r2_test = r2_score(y_test,y_test_pred)

r2_train, r2_test

(0.7670675608379445, 0.36194441651530607)

In [20]:
for i in range(len(y_test)):
    print(y_test.values[i], y_test_pred[i])

[6.71] 6.595694240152738
[7.119] 7.79737889331784
[6.347] 6.043534853434038
[6.057] 6.908664209510106
[6.108] 6.929055161903192
[6.468] 6.322069239669303
[7.301] 6.957084953823709
[7.26] 7.030581007791624
[6.921] 6.7277811198588315
[6.824] 6.563252633411593
[6.468] 6.884948482776735
[6.971] 7.215372291045695
[6.824] 6.859482267981206
[7.523] 7.4470653136216916
[7.337] 7.5691283188807565
[8.] 7.991488995585654
[7.62] 8.283952167396363
[7.222] 8.122202983156738
[8.046] 8.576014492649618


In [ ]:
################################

In [18]:
# mse = mean_squared_error(y_test,y_pred)
# rmse = np.sqrt(mse)
# print("RMSE :" , rmse)

In [19]:
# r2 = r2_score(y_test,pred)
# print("r2 :",r2)

In [20]:
# Testing 1 descriptor
# df = pd.DataFrame(list(zip(dfDesc.iloc[:,0], dtTrain.iloc[:,-1])), 
#                columns =[dfDesc.columns.values[1], dtTrain.columns.values[-1]])
# df

In [23]:
# # Get x & y for plot
# X = df.iloc[:,0]
# y = df.iloc[:,-1]
# X = X.to_numpy()
# X = X.reshape(-1,1)

In [ ]:
### SVM Here

# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1000, gamma=0.1, epsilon=.1)
svr_lin = SVR(kernel='linear', C=1000, gamma=0.1)
svr_poly = SVR(kernel='poly', C=1000, gamma=0.1, degree=0, epsilon=.1,
               coef0=1)

In [ ]:
plt.scatter(y_train,y_train_pred)